In [3]:
import os
import re
import unicodedata

import pandas as pd

target_data_dir = "../data/2_cleaned"
os.makedirs(target_data_dir, exist_ok=True)

In [4]:
filtered_out: dict[str, list[str]] = {
    "horoscope": [],
    "truncated_nordpresse": [],
    "censored": [],
    "quoted_only": [],
    "too_long": [],
    "non_french_like": [],
    "duplicates": [],
}


def clean_headline(headline: str, source: str) -> str | None:
    """Nettoie un titre en normalisant l'unicode, la ponctuation et la structure générale."""

    # 1) Normalisation Unicode
    headline = unicodedata.normalize("NFKC", headline)

    # 2) Filtre 1 : horoscope du Gorafi
    if headline.startswith("Horoscope du"):
        filtered_out["horoscope"].append(headline)
        return None

    # 3) Filtre 2 : titres tronqués de Nordpresse
    if source == "nordpresse" and headline.endswith("..."):
        filtered_out["truncated_nordpresse"].append(headline)
        return None

    # 4) Filtre 3 : titres contenant des "*" (= (auto)censure)
    if "*" in headline:
        filtered_out["censored"].append(headline)
        return None

    # 5) Filtre 4 : titres encadrés de guillemets (= propos rapportés)
    if re.match(r"^«[^«]+»$", headline):
        filtered_out["quoted_only"].append(headline)
        return None

    # 6) Nettoyage : suppression de l'initiale "Gorafi Magazine : "
    headline = re.sub(r"^Gorafi Magazine\s*.?\s*", "", headline)

    # 7) Nettoyage de la ponctuation
    headline = re.sub(r"\.$", "", headline)  # Suppression du point final
    headline = re.sub(
        r"([!?]){3,}", r"\1\1", headline
    )  # Réduction de la ponctuation expressive
    headline = re.sub(r"\.{2,}", "…", headline)  # Reformatage des points de suspension
    headline = re.sub(r"\s{2,}", " ", headline)  # Espaces blancs multiples consécutifs
    headline = re.sub(r"[“”«»]", '"', headline)  # Standardisation des guillemets
    headline = headline.replace("’", "'")  # Standardisation des apostrophes

    # 8) Filtre 5 : titre trop long
    if len(headline) > 150:
        filtered_out["too_long"].append(headline)
        return None

    # 9) Filtre 6 : titres (avec du) non français
    if re.match(r"[^A-Za-zÀ-ÖØ-öø-ÿæœ0-9\s'\"“”«»!?…:;’,/+&.–—\-()€$%°]{2,}", headline):
        filtered_out["non_french_like"].append(headline)
        return None

    return headline

In [ ]:
from pandas import DataFrame

sources = ["legorafi", "nordpresse", "lefigaro", "liberation"]

# Conteneur final pour fusion
cleaned_datasets: list[DataFrame] = []

for source in sources:
    df: DataFrame = pd.read_csv(f"../data/1_raw/{source}_headlines.csv")

    # Détection et suppression des doublons bruts (avant nettoyage complet)
    duplicated_mask = df.duplicated(subset=["headline"], keep=False)
    filtered_out["duplicates"].extend(df.loc[duplicated_mask, "headline"].tolist())
    df = df.drop_duplicates(subset=["headline"])

    # Application du nettoyage personnalisé des titres
    # → clean_headline retourne None si le titre courant doit être filtré
    df["cleaned"] = (
        df["headline"].astype(str).apply(lambda x: clean_headline(x, source))
    )
    df = df.dropna(subset=["cleaned"])

    df["source"] = source

    # Sauvegarde dataset nettoyé
    df[["cleaned", "is_satirical"]].to_csv(
        f"{target_data_dir}/{source}_headlines_cleaned.csv", index=False
    )

    # Ajout pour fusion
    cleaned_datasets.append(df[["cleaned", "is_satirical", "source"]])

# Fusion finale
merged_df = pd.concat(cleaned_datasets, ignore_index=True)
merged_df.to_csv("../data/cleaned_dataset.csv", index=False)

print(f"✅ Nettoyage et fusion terminés : {len(merged_df)} titres au total.")

# Reporting des titres filtrés
for key, items in filtered_out.items():
    print(f"\n🔍 {key} — {len(items)} titre(s) filtré(s) :")
    for title in items:
        print(" •", title)

✅ Nettoyage et fusion terminés : 9682 titres au total.

🔍 horoscope — 307 titre(s) filtré(s) :
 • Horoscope du 31 mars 2025
 • Horoscope du 24 mars 2025
 • Horoscope du 17 mars 2025
 • Horoscope du 10 mars 2025
 • Horoscope du 3 mars 2025
 • Horoscope du 24 février 2025
 • Horoscope du 17 février 2025
 • Horoscope du 10 février 2025
 • Horoscope du 3 février 2025
 • Horoscope du 27 janvier 2025
 • Horoscope du 20 janvier 2025
 • Horoscope du 13 janvier 2025
 • Horoscope du 6 janvier 2025
 • Horoscope du 16 décembre 2024
 • Horoscope du 9 décembre 2024
 • Horoscope du 2 décembre 2024
 • Horoscope du 25 novembre 2024
 • Horoscope du 18 novembre 2024
 • Horoscope du 12 novembre 2024
 • Horoscope du 4 novembre 2024
 • Horoscope du 28 octobre 2024
 • Horoscope du 21 octobre 2024
 • Horoscope du 14 octobre 2024
 • Horoscope du 7 octobre 2024
 • Horoscope du 30 septembre 2024
 • Horoscope du 23 septembre 2024
 • Horoscope du 16 septembre 2024
 • Horoscope du 9 septembre 2024
 • Horoscope du 2